# Predicción de cuotas de apuestas

En este ejercicio vamos a implementar una única red neuronal para predecir las cuotas de victoria, empate y derrota en partidos de fútbol

![img](https://www.ecestaticos.com/image/clipping/6192cd83066e196913e3045d412b937b/multan-a-la-casa-de-apuestas-digital-888-por-el-polemico-anuncio-de-carlos-sobera.jpg)

Los datos de cada casa de apuestas se encuentran en formato JSON dentro de la carpeta `data` con el nombre `bookmakerXXX.json`. Utiliza estos ficheros para crear un único dataframe con el que entrenar y probar el modelo.  
Por último, utiliza el fichero `predicciones.json` para analizar las cuotas de un único partido

In [1]:
# importar librerias.
import pandas as pd
import numpy as np
import tensorflow as tf
import json


from tensorflow import keras

In [2]:
# cargar datos

df = pd.read_json(r'C:\Users\ragod\Desktop\reposicion\ds_tb_part_21_09\03-Machine_Learning\Redes Neuronales\data\bookmaker17.json', orient ='index')
df2=df.transpose()
df2

,CoMatch,CoBookmaker,TsOdd,TsElapsed,VlProbH_ini,VlProbD_ini,CoMatchState,DsMatchState,VlGameTime,VlMinute,VlHomeScore,VlAwayScore,VlHomeRedCard,VlAwayRedCard,VlProbH,VlProbD,VlProbA
10578,1729828,17,2019-09-11 11:09:13,3913,0.45,0.27,3,2ST50,65,50.0,2,0,0,0,0.89,0.09,0.02
119314,1677221,17,2019-09-26 19:19:53,4613,0.52,0.27,3,2ST61,76,61.0,0,1,0,0,0.13,0.3,0.57
211966,1693901,17,2019-09-22 12:44:57,6117,0.29,0.3,3,2ST86,101,86.0,0,0,0,0,0.08,0.79,0.13
158689,1767673,17,2019-10-02 16:33:45,1965,0.51,0.25,1,1ST32,32,32.0,0,1,0,0,0.21,0.28,0.51
207804,1694621,17,2019-08-25 20:53:57,4917,0.55,0.27,3,2ST66,81,66.0,1,1,0,0,0.44,0.46,0.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
255370,1712573,17,2019-10-13 14:48:44,1124,0.34,0.3,1,1ST18,18,18.0,0,1,0,0,0.12,0.22,0.66
194318,1759475,17,2019-09-11 18:47:06,2766,0.38,0.29,1,1ST45+,46,46.0,1,0,0,0,0.71,0.22,0.08
269411,1693869,17,2019-09-01 10:31:41,1841,0.42,0.3,1,1ST30,30,30.0,0,0,0,0,0.42,0.36,0.22
190469,1722635,17,2019-09-01 10:36:57,2097,0.4,0.3,1,1ST34,34,34.0,0,0,0,0,0.4,0.36,0.23


In [3]:
import os
contenido = os.listdir(r'C:\Users\ragod\Desktop\reposicion\ds_tb_part_21_09\03-Machine_Learning\Redes Neuronales\data')
contenido

['.ipynb_checkpoints',
 'bookmaker17.json',
 'bookmaker22.json',
 'bookmaker274.json',
 'bookmaker282.json',
 'bookmaker285.json',
 'bookmaker30.json',
 'bookmaker327.json',
 'bookmaker37.json',
 'bookmaker385.json',
 'bookmaker42.json',
 'bookmaker6.json',
 'bookmaker83.json',
 'bookmaker84.json',
 'predicciones.json']

In [7]:
# del contenido[0]
# del contenido[-1]
contenido

['bookmaker17.json',
 'bookmaker22.json',
 'bookmaker274.json',
 'bookmaker282.json',
 'bookmaker285.json',
 'bookmaker30.json',
 'bookmaker327.json',
 'bookmaker37.json',
 'bookmaker385.json',
 'bookmaker42.json',
 'bookmaker6.json',
 'bookmaker83.json',
 'bookmaker84.json']

In [152]:
df_final=pd.DataFrame()
for file in contenido:
    with open ('data/'+ file) as json_file:
        data=json.load(json_file)
        df=pd.DataFrame.from_dict(data)
        df_final = pd.concat([df_final, df], axis = 0)


In [153]:
df_final

,CoMatch,CoBookmaker,TsOdd,TsElapsed,VlProbH_ini,VlProbD_ini,CoMatchState,DsMatchState,VlGameTime,VlMinute,VlHomeScore,VlAwayScore,VlHomeRedCard,VlAwayRedCard,VlProbH,VlProbD,VlProbA
10578,1729828,17,2019-09-11 11:09:13,3913,0.45,0.27,3,2ST50,65,50.0,2,0,0,0,0.89,0.09,0.02
119314,1677221,17,2019-09-26 19:19:53,4613,0.52,0.27,3,2ST61,76,61.0,0,1,0,0,0.13,0.30,0.57
211966,1693901,17,2019-09-22 12:44:57,6117,0.29,0.30,3,2ST86,101,86.0,0,0,0,0,0.08,0.79,0.13
158689,1767673,17,2019-10-02 16:33:45,1965,0.51,0.25,1,1ST32,32,32.0,0,1,0,0,0.21,0.28,0.51
207804,1694621,17,2019-08-25 20:53:57,4917,0.55,0.27,3,2ST66,81,66.0,1,1,0,0,0.44,0.46,0.10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100949,1724257,84,2019-09-13 21:32:07,5347,0.69,0.19,3,2ST74,89,74.0,1,0,0,0,0.85,0.13,0.02
99925,1677253,84,2019-10-20 14:47:59,2819,0.32,0.29,1,1ST45+,46,46.0,0,1,0,0,0.12,0.26,0.62
71473,1724595,84,2019-09-08 18:37:50,5690,0.40,0.28,3,2ST79,94,79.0,2,2,0,0,0.02,0.16,0.81
232985,1677256,84,2019-10-19 20:34:38,5558,0.16,0.22,3,2ST77,92,77.0,1,0,0,1,0.74,0.21,0.05


In [154]:
#cargamos prediccion
with open ('data/'+ 'predicciones.json') as json_file:
               data=json.load(json_file)
               df_prediccion = pd.DataFrame.from_dict(data)
               



In [155]:
df_prediccion

,TsOdd,TsElapsed,VlProbH_ini,VlProbD_ini,CoMatchState,DsMatchState,VlGameTime,VlMinute,VlHomeScore,VlAwayScore,VlHomeRedCard,VlAwayRedCard,VlProbH,VlProbD,VlProbA,NaMatch
0,2019-10-01 19:00:50,50,0.6,0.25,1,1ST0,0,0.0,0,0,0,0,0.60,0.25,0.15,Celta vs Valladolid
1,2019-10-01 19:09:51,591,0.6,0.25,1,1ST9,9,9.0,0,0,0,0,0.52,0.30,0.18,Celta vs Valladolid
2,2019-10-01 20:48:32,6452,0.6,0.25,3,2ST90+,107,92.0,1,0,0,0,0.95,0.05,0.00,Celta vs Valladolid
3,2019-10-01 20:33:53,5573,0.6,0.25,3,2ST77,92,77.0,1,0,0,0,0.87,0.12,0.01,Celta vs Valladolid
4,2019-10-01 20:04:27,3807,0.6,0.25,3,2ST48,63,48.0,1,0,0,0,0.79,0.16,0.05,Celta vs Valladolid
5,2019-10-01 20:38:56,5876,0.6,0.25,3,2ST82,97,82.0,1,0,0,0,0.89,0.10,0.01,Celta vs Valladolid
6,2019-10-01 19:12:51,771,0.6,0.25,1,1ST12,12,12.0,0,0,0,0,0.52,0.30,0.18,Celta vs Valladolid
7,2019-10-01 19:31:10,1810,0.6,0.25,1,1ST30,30,30.0,1,0,0,0,0.76,0.18,0.06,Celta vs Valladolid
8,2019-10-01 20:48:41,6461,0.6,0.25,3,2ST90+,107,92.0,1,0,0,0,0.94,0.06,0.00,Celta vs Valladolid
9,2019-10-01 20:34:05,5585,0.6,0.25,3,2ST78,93,78.0,1,0,0,0,0.87,0.12,0.01,Celta vs Valladolid


In [156]:
df_final.describe()

,CoMatch,CoBookmaker,TsElapsed,VlProbH_ini,VlProbD_ini,CoMatchState,VlGameTime,VlMinute,VlHomeScore,VlAwayScore,VlHomeRedCard,VlAwayRedCard,VlProbH,VlProbD,VlProbA
count,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,9694.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,1.705711e+06,198.140800,2985.280300,0.431227,0.277829,1.852400,49.26420,42.741593,0.446900,0.392400,0.039700,0.028900,0.393787,0.329939,0.276252
std,3.046140e+04,143.794335,1979.310646,0.131598,0.039863,0.979035,32.98975,26.786407,0.673369,0.635344,0.203784,0.169315,0.243532,0.162054,0.218576
min,1.676673e+06,6.000000,2.000000,0.100000,0.060000,1.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000
25%,1.677186e+06,42.000000,1386.500000,0.340000,0.260000,1.000000,23.00000,22.000000,0.000000,0.000000,0.000000,0.000000,0.210000,0.220000,0.120000
50%,1.694637e+06,282.000000,2354.000000,0.420000,0.290000,1.000000,39.00000,37.000000,0.000000,0.000000,0.000000,0.000000,0.360000,0.310000,0.230000
75%,1.725137e+06,327.000000,5148.000000,0.510000,0.310000,3.000000,85.00000,71.000000,1.000000,1.000000,0.000000,0.000000,0.552500,0.380000,0.350000
max,1.778095e+06,385.000000,6559.000000,0.920000,0.350000,4.000000,109.00000,94.000000,4.000000,4.000000,2.000000,2.000000,0.970000,0.920000,0.990000


In [157]:
#eliminamos CoMatch, CoBOOKmaker, CoMatchState
df_final2=df_final.drop(['CoMatch', 'CoBookmaker', 'CoMatchState','DsMatchState','TsOdd'],axis =1)

In [158]:
# Separamos en funcion de los partidos. Clculamos cuantos partidos hay.
num_partidos_train = len(df_final['CoMatch'].unique())

In [159]:
num_partidos_train

917

In [160]:
np.random.seed(50)

comatches=df_final.CoMatch.unique()
len(comatches)

train_len =int(0.85*df_final.CoMatch.nunique())

np.random.shuffle(comatches)

comatches_train,comatches_test = comatches[:train_len], comatches[train_len:]

In [161]:
# Cramos el df_train

li_1=[]
li_2=[]
for k in comatches_train:
    li_1 =df_final.loc[df_final.CoMatch == k ].index
    for l in li_1:
        li_2.append(l)
df_train = df_final2.loc[li_2]

X_train = df_train.iloc[:,:-3]
y_train = df_train.iloc[:,-3:]

In [162]:
# Cramos el df_test

li_1=[]
li_2=[]
for k in comatches_test:
    li_1 =df_final.loc[df_final.CoMatch == k ].index
    for l in li_1:
        li_2.append(l)
df_test = df_final2.loc[li_2]

In [163]:
# Cramos el df_test

li_1=[]
li_2=[]
for k in comatches_test:
    li_1 =df_final.loc[df_final.CoMatch == k ].index
    for l in li_1:
        li_2.append(l)
df_train = df_final2.loc[li_2]

X_test = df_train.iloc[:,:-3]
y_test = df_train.iloc[:,-3:]

In [165]:
#Normalizamos variables de X_train y X_test

from sklearn.preprocessing import MinMaxScaler
x_MinMax = MinMaxScaler()
y_MinMax = MinMaxScaler()

x = x_MinMax.fit_transform(X_train)
y = y_MinMax.fit_transform(y_train)


In [ ]:
import tensorflow as tf
from tensorflow import keras